**Processing K's Images**<br>


In [53]:
import syotil

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os
from skimage import img_as_uint

import tensorflow as tf

from deepcell_toolbox.utils import resize, tile_image, untile_image

In [59]:
import glob
INPUT_PATH="images/training/"
#INPUT_PATH="images/training_512x512/"
FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
print(len(FILENAMES))
print(FILENAMES)

7
['images/training/M872956_JML_Position9_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD3_train_img.png', 'images/training/M926910_CFL_Position7_CD3_train_img.png', 'images/training/M926910_CFL_Position13_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD8_train_img.png', 'images/training/M872956_JML_Position10_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD4_train_img.png']


In [60]:
imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
X_train = tf.stack(imgs)
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)

masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
y_train = tf.stack(masks)
y_train = np.expand_dims(y_train, axis=-1)
print(y_train.shape)

# np.savez("K_training_data", X=X_train, y=y_train) # objects to save need to be key value pairs
# train_dict, test_dict = get_data("K_training_data.npz", test_size=test_size, seed=seed)

(7, 1040, 1159, 1)
(7, 1040, 1159, 1)


In [61]:
# resize image
mpp_select=1.3 # based on DeepCell_tn_nuclear.ipynb, the best value is 1.2, but 1.3 makes scale factor an integer
shape = X_train.shape
scale_factor = mpp_select / 0.65
print(scale_factor)
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train_resized = resize(X_train, new_shape, data_format='channels_last')
y_train_resized = resize(y_train, new_shape, data_format='channels_last')
print(X_train_resized.shape)
print(y_train_resized.shape)

2.0
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)


In [63]:
# write resized images and masks. saved to images/training_resized
for i in range(7):
    fname=FILENAMES[i].replace("_img","_resized_img").replace("training","training_resized")
    io.imsave(fname, X_train_resized[i,:,:,0], check_contrast=False)

Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 250]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.


In [66]:
# previous warnings are okay because:
np.max(X_train_resized[i,:,:,0])

254

In [68]:
for i in range(7):
    fname=FILENAMES[i].replace("_img","_resized_masks").replace("training","training_resized")
    io.imsave(fname, img_as_uint(y_train_resized[i,:,:,0]), check_contrast=False)

In [69]:
print(X_train_resized.shape)
print(y_train_resized.shape)

x_starts=[0,400,400*2,400*3,X_train_resized.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,512,512*2,512*3,X_train_resized.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
[0, 400, 800, 1200, 1568]
[512, 912, 1312, 1712, 2080]
[0, 512, 1024, 1536, 1806]
[512, 1024, 1536, 2048, 2318]


In [72]:
for i in range(len(x_starts)):
    for j in range(len(y_starts)):
        sub_X = X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        sub_y = y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        for idx, CURR_IM_NAME in enumerate(FILENAMES):
            fname=CURR_IM_NAME.replace("training","training_resized_512x512").replace("train_img","train{}_img".format(j+i*len(y_starts)))
            io.imsave(fname, img_as_ubyte(sub_X[idx,...]), check_contrast=False) # use ubyte to suppress warnings
            fname=CURR_IM_NAME.replace("training","training_resized_512x512").replace("train_img","train{}_masks".format(j+i*len(y_starts)))
            io.imsave(fname, img_as_uint(sub_y[idx,...]), check_contrast=False)
